Netlogo is often run from its own GUI, but it can also be run 'headless'; that is to say, run from the command line or from within another program, in the background. That is what we are doing here from this notebook. We will use the [NLRX package](https://nldoc.github.io/nlrx/) to interface between R and Netlogo. This gives us the advantage of gathering data for analysis within R while running models written in the nlogo language. We can write models using Python or other langugages, but Netlogo's language is particularly well-suited for agent modeling. This notebook recreates the spatial output walkthrough at [nldoc.github.io site](https://nldoc.github.io/nlrx/articles/articles/spatial-output.html).

In [56]:
library("magrittr")

In [57]:
# this package was installed during the creation of the binder
# if you get an error running this line:
library("nlrx", lib.loc="/srv/rlibs")

# insert the following line into a block *above* this one, remove the #, and then run it:
# devtools::install_github("nldoc/nlrx")

In [58]:
# this package helps with running the simulation.
library("furrr")

We create a variable, `nl`, that tells the package where to find Netlogo (which we installed when we built this binder), where to find the model, written in `nlogo` code, and how much memory to set aside for the java virtual machine that will run the model.

In [61]:
nl <- nl(nlversion = "6.0.4",
         nlpath = "/home/jovyan/NetLogo 6.0.4/",
         modelpath = "/home/jovyan/NetLogo 6.0.4/app/models/Sample Models/Biology/Wolf Sheep Predation.nlogo",
         jvmmem = 1024)

Now, let's define the experiment that we are going to run, by specifying all of the various parameters available in the model.

In [62]:
nl@experiment <- experiment(expname = "nlrx_spatial",
                            outpath="out/",
                            repetition = 1,      
                            tickmetrics = "true",
                            idsetup = "setup",   
                            idgo = "go",         
                            idfinal = NA_character_,  
                            idrunnum = NA_character_,
                            runtime = 100,
                            evalticks = seq(1,100),
                            metrics = c("count sheep","count wolves"),
                            metrics.turtles = c("who", "pxcor", "pycor", "breed"),
                            metrics.patches = c("pxcor", "pycor", "pcolor"),
                            constants = list("model-version" = "\"sheep-wolves-grass\"",
                                             'initial-number-sheep' = 100,
                                             'initial-number-wolves' = 50,
                                             "grass-regrowth-time" = 30,
                                             "sheep-gain-from-food" = 4,
                                             "wolf-gain-from-food" = 20,
                                             "sheep-reproduce" = 4,
                                             "wolf-reproduce" = 5,
                                             "show-energy?" = "false")
                            )

From the nlrx package documentation: "While the experiment defines the variables and specifications of the model, the simulation design creates a parameter input table based on these model specifications and the chosen simulation design method. nlrx provides a bunch of different simulation designs, such as full-factorial, latin-hypercube, sobol, morris and eFast. A simulation design is attached to a nl object by using one of these simdesign functions:"

In [63]:
nl@simdesign <- simdesign_simple(nl=nl,
                                 nseeds=1)


Creating simple simulation design


This experiment will run for 100 ticks and collects all metrics, metrics.turtles and metrics.patches on each tick. Thus, running run_nl_all() will report a tibble containing all metrics, metrics.turtles and metrics.patches. However, because the spatial metrics contain more than one value, these datasets are stored as lists inside the output tibble. This lists already contain all measured agent metrics and can for example be used to analyse distributions of these variables for specific agent groups.

In case of spatial data containing coordinates (pxcor/pycor for patches or pxcor/pycor and/or xcor/ycor for turtles), nlrx provides a function to transform the measured output into spatial objects. In order to use the function get_nl_spatial() you first have to attach your simulation output results to the nl object:

In [64]:
future::plan(multisession)
results %<-% run_nl_all(nl = nl, cleanup = "all")

In [65]:
# Attach results to nl object:
setsim(nl, "simoutput") <- results

# Report spatial data:
results_spatial <- get_nl_spatial(nl,
                                  turtles = TRUE,
                                  patches = TRUE,
                                  turtle_coords = "px",
                                  format="spatial")

ERROR: Error: Expressions are currently not supported in `rename()`


The get_nl_spatial() function uses 4 function parameters:

turtles - TRUE/FALSE, if true the function will transform the measured metrics.turtles to spatial sf objects.
patches - TRUE/FALSE, if true the function will transform the measured metrics.patches to spatial raster objects. In case there are several patches variables besides the coordinates, the result will be a rasterstack containing one raster for each patch variable
turtle_coords - “px”/“x”, because turtle coordinates can be measured as pxcor/pycor or xcor/ycor this string defines which coordinates are used for creating the turtle sf objects. For instance, you can measure both coordinate pairs by entering them into metrics.turtles, and then decide for one type of coordinates to create spatial objects.
format - “spatial”/“tibble”, the “spatial” option will report the generated output as tibble containing the spatial objects (raster/rasterStack, sf). You can also choose format “tibble” report the spatial objects as a long format tibble, that can be used for plotting landscapes and turtles with ggplot2.
